In [2]:
import os
import librosa
import librosa.display as dsp
from IPython.display import Audio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

ROOT = r"E:월간 데이콘 음성 감정 인식"

In [3]:
train_df = pd.read_csv(os.path.join(ROOT, "train.csv"))
train_df["path"] = train_df["path"].apply(lambda x: os.path.join(ROOT, x[2:].replace("/", "\\")))
print(train_df.shape)
train_df.head()

(5001, 3)


,id,path,label
0,TRAIN_0000,E:월간 데이콘 음성 감정 인식\train\TRAIN_0000.wav,1
1,TRAIN_0001,E:월간 데이콘 음성 감정 인식\train\TRAIN_0001.wav,2
2,TRAIN_0002,E:월간 데이콘 음성 감정 인식\train\TRAIN_0002.wav,4
3,TRAIN_0003,E:월간 데이콘 음성 감정 인식\train\TRAIN_0003.wav,5
4,TRAIN_0004,E:월간 데이콘 음성 감정 인식\train\TRAIN_0004.wav,4


In [11]:
import random
import torch
import torch.nn as nn
from tqdm.auto import tqdm
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from transformers import AutoModelForAudioClassification, Wav2Vec2FeatureExtractor

from sklearn.model_selection import train_test_split

In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [9]:
CFG = {
    'SR':16_000,
    'SEED':42,
    'BATCH_SIZE':8, 
    'TOTAL_BATCH_SIZE':32, 
    'EPOCHS':1,
    'LR':1e-4,
}
MODEL_NAME = "facebook/wav2vec2-base"

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-Processing

In [10]:
train_df, valid_df = train_test_split

,id,path,label
0,TRAIN_0000,E:월간 데이콘 음성 감정 인식\train\TRAIN_0000.wav,1
1,TRAIN_0001,E:월간 데이콘 음성 감정 인식\train\TRAIN_0001.wav,2
2,TRAIN_0002,E:월간 데이콘 음성 감정 인식\train\TRAIN_0002.wav,4
3,TRAIN_0003,E:월간 데이콘 음성 감정 인식\train\TRAIN_0003.wav,5
4,TRAIN_0004,E:월간 데이콘 음성 감정 인식\train\TRAIN_0004.wav,4
...,...,...,...
4996,TRAIN_4996,E:월간 데이콘 음성 감정 인식\train\TRAIN_4996.wav,5
4997,TRAIN_4997,E:월간 데이콘 음성 감정 인식\train\TRAIN_4997.wav,0
4998,TRAIN_4998,E:월간 데이콘 음성 감정 인식\train\TRAIN_4998.wav,1
4999,TRAIN_4999,E:월간 데이콘 음성 감정 인식\train\TRAIN_4999.wav,1
